In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random

2024-06-26 09:28:53.485636: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
fls = ['bestflats_ex - bestflats_ex.csv.csv',
'bitkogan - bitkogan.csv.csv',
'nlevshitstelegram - nlevshitstelegram.csv.csv',
'promtyu - promtyu.csv.csv', 
'rbc_news - rbc_news.csv.csv',
'pikabu - pikabu.csv.csv',
'sale_caviar - sale_caviar.csv.csv']

In [3]:
from datetime import timedelta
import os
ful = pd.DataFrame()
for i in fls:
    print(i.split(' ')[0])
    df = pd.read_csv(i)
    df['time'] = pd.to_datetime(df['time'])
    filename = []
    for j in range(len(df)):
        if str(df.time.iloc[j]) + '.jpg' in os.listdir(i.split(' ')[0]):
            filename.append(i.split(' ')[0]+ '/' + str(df.time.iloc[j]) + '.jpg')
        else:
            if str(df.time.iloc[j] + timedelta(seconds = 1)) + '.jpg' in os.listdir(i.split(' ')[0]):
                filename.append(i.split(' ')[0]+ '/' + str(df.time.iloc[j] + timedelta(seconds = 1)) + '.jpg')
            elif str(df.time.iloc[j] + timedelta(seconds = -1)) + '.jpg' in os.listdir(i.split(' ')[0]):
                
                filename.append(i.split(' ')[0]+ '/' + str(df.time.iloc[j] + timedelta(seconds = -1)) + '.jpg')
            else:
                filename.append(None)
    df['filename'] = filename
    ful = pd.concat([ful, df])

bestflats_ex
bitkogan
nlevshitstelegram
promtyu
rbc_news
pikabu
sale_caviar


In [6]:
no_pics = pd.concat([pd.read_csv('headlines_for_traders - df.csv.csv'), pd.read_csv('fatcat - df.csv.csv')])
no_pics['time'] = None
no_pics['filename'] = None
ful = pd.concat([ful, no_pics])

/tmp/ipykernel_1162241/1314617571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ful = pd.concat([ful, no_pics])


In [7]:
ful = ful.dropna(subset = ['message', 'ad'])
ful = ful.sample(frac = 1, random_state = 1)

In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:
df = pd.read_csv('text.csv')

In [12]:
import random
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)
import pickle

oov_tok = "<OOV>"
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(df.text)
max_len_text = 1000 
text_sequences = tokenizer.texts_to_sequences(ful['message'])
text_data = pad_sequences(text_sequences, maxlen=max_len_text)


image_data = []
for filename in ful['filename']:
    if filename is not None:
        image_path = os.path.join(filename)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array /= 255.0  
        image_data.append(img_array)
    else:
        image_data.append(np.zeros((224, 224, 3)))   
image_data = np.array(image_data)


In [13]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


inputs_text = Input(shape=(max_len_text,))
inputs_image = Input(shape=(224, 224, 3))

embedding_dim = 8  
text_model = Embedding(vocab_size, embedding_dim, input_length=max_length)(inputs_text)
text_model = GlobalAveragePooling1D()(text_model)
text_model = Dense(200, activation='relu')(text_model)
text_model = Dense(100, activation='relu')(text_model)
text_model = Dense(50, activation='relu')(text_model)
text_model = Dense(24, activation='relu')(text_model)
text_model = Dense(12, activation='relu')(text_model)

image_model = Conv2D(32, (3, 3), activation='relu')(inputs_image)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(64, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(128, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Flatten()(image_model)
image_model = Dense(256, activation='relu')(image_model)
image_model = Dropout(0.5)(image_model)

concatenated = Concatenate()([text_model, image_model])
output = Dense(1, activation='sigmoid')(concatenated)
    
model = Model(inputs=[inputs_text, inputs_image], outputs=output)
Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])
model.load_weights('mm_weights.weights.h5')

X_text_train, X_text_test, X_image_train, X_image_test, y_train, y_test = train_test_split(
    text_data, image_data, ful['ad'], test_size=0.2, random_state=42)


model.fit([X_text_train, X_image_train], y_train, batch_size=32, epochs=6, validation_split=0.2)

loss, accuracy = model.evaluate([X_text_test, X_image_test], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/6
117/117 [==============================] - 99s 817ms/step - loss: 1.0895 - accuracy: 0.8761 - val_loss: 0.9750 - val_accuracy: 0.8937
Epoch 2/6
117/117 [==============================] - 95s 809ms/step - loss: 0.8462 - accuracy: 0.8936 - val_loss: 0.8096 - val_accuracy: 0.9076
Epoch 3/6
117/117 [==============================] - 93s 794ms/step - loss: 0.7002 - accuracy: 0.9062 - val_loss: 0.6655 - val_accuracy: 0.9098
Epoch 4/6
117/117 [==============================] - 98s 837ms/step - loss: 0.5803 - accuracy: 0.9097 - val_loss: 0.5465 - val_accuracy: 0.9141
Epoch 5/6
117/117 [==============================] - 95s 816ms/step - loss: 0.4833 - accuracy: 0.9081 - val_loss: 0.4703 - val_accuracy: 0.9162
Epoch 6/6
37/37 [==============================] - 7s 192ms/step - loss: 0.3834 - accuracy: 0.9270
Test Loss: 0.3834344446659088
Test Accuracy: 0.9269759654998779
